In [247]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from re import search as re_search
from re import DOTALL
from retrying import retry

In [248]:
dest_website = 'https://crautos.com/autosusados/'
trans_re = r'^(?P<engine>.*?)\|(?P<transmission>.*?)\|(?P<fuel>.*)'
vechicle_info_re = r'(?P<name>\w.*?)\s*\n.*?•\s*(?P<passenger>\w.*?)\s*\n.*?(?P<year>\w.*?)\s*$'

In [249]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [250]:
# get test site
driver.get(dest_website)
sleep(3)

In [251]:
# find search button
search_button = driver.find_element(By.XPATH, '//input[@type="submit"]')
# click search button
search_button.click()
sleep(3)

In [267]:
def get_price_number(price_txt):
    price_re = r'\d+'
    price_txt = price_txt.replace(',', '')
    price_txt = re_search(price_re, price_txt).group(0)
    return price_txt


def add_specs_to_dict(invetory, spec_type, res_dict):
    options_spec = f'./table[@class="options-{spec_type}"]/tbody/tr'
    spec_mytext1 = f'./td[@class="{spec_type} mytext1"]'
    for spec in inventory.find_elements(By.XPATH, options_spec):
        #print(primary_spec.get_attribute('innerText'))
        try:
            item = spec.find_element(By.XPATH, spec_mytext1)
            item = item.get_attribute('innerText')
            #clean item name
            item = item.replace(':', '').strip()
            item_val = spec.find_element(By.XPATH, './td[@class="spec"]')
            item_val = item_val.get_attribute('innerText')
            item_val = item_val.strip()
            res_dict[item] = item_val
        except NoSuchElementException:
            continue


def add_specs_to_dict1(invetory, spec_type, res_dict):
    options_txt = f'options-{spec_type}'
    spec_mytext1 = f'./td[@class="{spec_type} mytext1"]'
    #for spec in inventory.find_elements(By.CLASS_NAME, options_txt):
    for spec in inventory.find_elements(By.XPATH, options_txt):
        #print(primary_spec.get_attribute('innerText'))
        try:
            item = spec.find_element(By.XPATH, spec_mytext1)
            item = item.get_attribute('innerText')
            #clean item name
            item = item.replace(':', '').strip()
            item_val = spec.find_element(By.XPATH, './td[@class="spec"]')
            item_val = item_val.get_attribute('innerText')
            item_val = item_val.strip()
            res_dict[item] = item_val
        except NoSuchElementException:
            continue


# define method to parse page
def get_car_dict(inventory):
    res = {}
    # title
    title_info = inventory.find_element(By.XPATH, './div[@class="title"]')
    title_info = title_info.get_attribute('innerText').strip()
    res['titulo'] = title_info
    print(res)
    # price colones
    price_colon = inventory.find_element(By.XPATH, './/div[@class="pricetext"]')
    price_colon = price_colon.get_attribute('innerText').strip()
    price_colon = get_price_number(price_colon)
    res['precio colones'] = price_colon
    print(res)
    # price dollars
    price_dollar = inventory.find_element(By.XPATH, './/div[@class="pricedollars"]')
    price_dollar = price_dollar.get_attribute('innerText').strip()
    price_dollar = get_price_number(price_dollar)
    res['precio dolares'] = price_dollar
    print(res)
    # primary specs
    add_specs_to_dict(inventory, 'primary', res)
    print(res)
    # secondary specs
    add_specs_to_dict(inventory, 'secondary', res)
    print(res)
    return res


def retry_on_stale(ex):
    return isinstance(ex, StaleElementReferenceException)


def refresh_page(driver):
    driver.refresh()
    sleep(5)


def get_car_info_list(driver):
    car_info_list = []
    for inventory in driver.find_elements(By.XPATH, '//a[@class="inventory"]'):
        car_info_list.append(get_car_dict(inventory))
    return car_info_list


def retry_get_car_info(driver):
    retries = 2
    car_info_list = []
    print('1')
    while retries >= 0:
        try:
            print('2')
            car_info_list = get_car_info_list(driver)
            print('3')
            break
        except StaleElementReferenceException:
            print('4')
            refresh_page(driver)
            print('5')
            retries = retries - 1
            if retries < 0:
                raise
            sleep(5)
    return car_info_list


In [268]:
car_info_list = retry_get_car_info(driver)
car_info_list

1
2
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000', 'precio dolares': '2385'}
4
5
2
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000', 'precio dolares': '2385'}
4
5
2
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000', 'precio dolares': '2385'}
4
5


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=111.0.5563.64)
Stacktrace:
0   chromedriver                        0x0000000104746428 chromedriver + 4899880
1   chromedriver                        0x00000001046c3a23 chromedriver + 4364835
2   chromedriver                        0x000000010430dbf6 chromedriver + 474102
3   chromedriver                        0x0000000104311c94 chromedriver + 490644
4   chromedriver                        0x0000000104311876 chromedriver + 489590
5   chromedriver                        0x0000000104311d1c chromedriver + 490780
6   chromedriver                        0x000000010435110c chromedriver + 749836
7   chromedriver                        0x0000000104351751 chromedriver + 751441
8   chromedriver                        0x0000000104343b63 chromedriver + 695139
9   chromedriver                        0x000000010437758d chromedriver + 906637
10  chromedriver          

In [261]:
for inventory in driver.find_elements(By.XPATH, '//a[@class="inventory"]'):
    title_info = inventory.find_element(By.XPATH, './div[@class="title"]')
    vehicle_info = title_info.get_attribute('innerText')
    print(vehicle_info)
    # primary specs
    primary_specs = {}
    for primary_spec in inventory.find_elements(By.XPATH, './table[@class="options-primary"]/tbody/tr'):
        #print(primary_spec.get_attribute('innerText'))
        try:
            primary_item = primary_spec.find_element(By.XPATH, './td[@class="primary mytext1"]')
            primary_item = primary_item.get_attribute('innerText')
            primary_item_val = primary_spec.find_element(By.XPATH, './td[@class="spec"]')
            primary_item_val = primary_item_val.get_attribute('innerText')
            primary_specs[primary_item] = primary_item_val
        except NoSuchElementException:
            continue
    print(primary_specs)
    # secondary specs
    secondary_specs = {}
    for secondary_spec in inventory.find_elements(By.XPATH, './table[@class="options-secondary"]/tbody/tr'):
        #print(primary_spec.get_attribute('innerText'))
        try:
            secondary_item = secondary_spec.find_element(By.XPATH, './td[@class="secondary mytext1"]')
            secondary_item = secondary_item.get_attribute('innerText')
            secondary_item_val = secondary_spec.find_element(By.XPATH, './td[@class="spec"]')
            secondary_item_val = secondary_item_val.get_attribute('innerText')
            secondary_specs[secondary_item] = secondary_item_val
        except NoSuchElementException:
            continue
    print(secondary_specs)
    # precio
    price = inventory.find_element(By.XPATH, './/div[@class="pricetext"]')
    print(price.get_attribute('innerText').strip())
    # price dollars
    price_dollar = inventory.find_element(By.XPATH, './/div[@class="pricedollars"]')
    print(price_dollar.get_attribute('innerText').strip())

2012 Volkswagen AMAROK  • 5 Pas.
{'Combustible:': 'Diesel', 'Cilindrada:': '2000 cc', 'Transmisión:': 'Manual', 'Estilo:': 'Pickup 4x4'}
{'Color Exterior:': 'GRIS RATÓN', 'Color Interior:': 'NEGRO Y GRIS', 'Placa:': '\n                                          \n                                              Termina en 6 \n                                        ', 'Provincia:': 'Alajuela'}
¢ 1,300,000
($ 2,385)*
1987 Toyota TERCEL  • 4 Pas.
{'Combustible:': 'Gasolina', 'Cilindrada:': '1500 cc', 'Transmisión:': 'Manual', 'Estilo:': 'Hatchback'}
{'Color Exterior:': 'DORADO', 'Color Interior:': 'GRIS', 'Placa:': '\n                                          \n                                              Termina en 9 \n                                        ', 'Provincia:': 'Alajuela'}
¢ 1,000,000
($ 1,835)*
1989 Nissan SENTRA B12
{'Combustible:': 'Gasolina', 'Cilindrada:': '1600 cc', 'Transmisión:': 'Manual', 'Estilo:': 'Sedán'}
{'Color Exterior:': 'CELESTE', 'Color Interior:': 'GRIS OSC

In [176]:
# find next button and go to next page
next_button = driver.find_element(By.CSS_SELECTOR, '.fa-angle-right')
next_button.click()
sleep(3)

In [238]:
def setup_extraction(driver):
    # get test site
    driver.get(dest_website)
    sleep(10)
    # find search button
    search_button = driver.find_element(By.XPATH, '//input[@type="submit"]')
    # click search button
    search_button.click()
    sleep(10)


def go_to_next_page(driver):
    try:
        # find next button and go to next page
        next_button = driver.find_element(By.CSS_SELECTOR, '.fa-angle-right')
        next_button.click()
        sleep(10)
        return True
    except NoSuchElementException:
        return False

In [266]:
res = []
#with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    #setup_extraction(driver)
print('Page 0')
res = get_car_info_list(driver)
count = 1
while (go_to_next_page(driver)):
    print(f'Page {count}')
    res += get_car_info_list(driver)
    count += 1
    if count >= 10:
        break
res

Page 0
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000'}
{'titulo': '1988 Toyota COROLLA FX \xa0•\xa05 Pas.', 'precio colones': '1300000', 'precio dolares': '2385'}


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=111.0.5563.64)
Stacktrace:
0   chromedriver                        0x0000000104746428 chromedriver + 4899880
1   chromedriver                        0x00000001046c3a23 chromedriver + 4364835
2   chromedriver                        0x000000010430dbf6 chromedriver + 474102
3   chromedriver                        0x0000000104311c94 chromedriver + 490644
4   chromedriver                        0x0000000104311876 chromedriver + 489590
5   chromedriver                        0x0000000104311d1c chromedriver + 490780
6   chromedriver                        0x000000010435110c chromedriver + 749836
7   chromedriver                        0x0000000104351751 chromedriver + 751441
8   chromedriver                        0x0000000104343b63 chromedriver + 695139
9   chromedriver                        0x000000010437758d chromedriver + 906637
10  chromedriver          